<a href="https://colab.research.google.com/github/quanvu0996/ds_test/blob/main/SOLUTION_DS_TEST_PRACTICE_T3_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ĐÁNH GIÁ NĂNG LỰC KỸ SƯ KHOA HỌC DỮ LIỆU THÁNG 03 - 2022
#### BÀI THỰC HÀNH (ĐÁP ÁN THAM KHẢO)

---

Ngày test: 25/03/2022
Thời gian làm bài: 75 phút, từ 10:00 đến 11:15
(+ 5 phút thiết lập môi trường và nộp bài)


**Quy định:**
- Không trao đổi trong suốt quá trình làm bài
- Record màn hình làm bài và share lại cho quanvh8 sau khi kết thúc thời gian test
- Được sử dụng tài liệu và tra cứu internet
- Sau khi hoàn thành, tải notebook, gửi về email: quanvh8@viettel.com.vn

**Hướng dẫn làm bài:**

* Chạy toàn bộ lệnh trong phần 0. CHUẨN BỊ MÔI TRƯỜNG
* Làm theo hướng dẫn tại các câu và CHỈ SỬA các đoạn code nằm giữa các thẻ:

      # YOUR CODE START
      # / END YOUR CODE

*Lưu ý: hầu hết câu hỏi không có tính tuần tự (không cần hoàn thành câu trước để làm câu sau)*

*Đề thi có một số testcase hỗ trợ DS kiểm tra kết quả, nhưng đây không phải điểm số. Điểm số được giám khảo chấm và gửi lại sau.*



---

### 0. CÀI ĐẶT MÔI TRƯỜNG
Guide: Chạy toàn bộ các lệnh trong phần này

In [ ]:
!pip install pyspark

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import gzip
import shutil
import requests 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

import pyspark
from pyspark.sql import functions as F

In [ ]:
def data_downloader(file_name):
    file_url = 'https://datasets.imdbws.com/'+file_name
    file_path = "/content/"+file_name
    
    r = requests.get(file_url, stream = True, verify=False) 
    
    with open( file_path, "wb") as file: 
        for block in r.iter_content(chunk_size = 1024*10): 
            if block: 
                file.write(block)
    print("Loaded "+file_name+" to local path: "+file_path)

# TEST CASE
class TEST():
    def test_1(df):
        tc1 = (df.columns ==['tconst', 'averageRating', 'numVotes'])
        tc2 = (df.count() >= 1227518)
        if (tc1 and tc2): print("PASS")
        else: print('FAIL')

    def test_2(df):
        tc1 = (df.columns == ['titleId','ordering','title','region','language','types','attributes','isOriginalTitle'])
        tc2 = (df.count() >= 31450282)
        if (tc1 and tc2): print("PASS")
        else: print('FAIL')

    def iia_schema(df_pred):
      tc1 = (df_pred.columns == ['tconst', 'pred'])
      tc2 = (df_pred.count() == test_ratings.count() )
      if tc1 and tc2 :
        print("PASS: dataframe đầu ra đúng định dạng")
      else:
        print("FAIL: dataframe đầu ra không đúng định dạng, đ/c kiểm tra lại")
      
      tc3 = (df_pred.join(test_ratings, on = 'tconst').count() == test_ratings.count())
      
      if tc3 :
        print("PASS: dataframe đầu ra đúng số lượng bản ghi")
      else:
        print("FAIL: dataframe đầu ra không đúng số lượng bản ghi, đ/c kiểm tra lại")
        
    def mae_iia(df_pred):
      result = df_pred.join(test_ratings, on = 'tconst')
      result = result.withColumn('ae', F.abs(F.col("pred")-F.col("averageRating")))
      result.selectExpr("avg(ae) AS mae").show()

    def iib_schema(df_pred):
      tc1 = ( list(df_pred.columns) == ['tconst', 'pred'])
      tc2 = (df_pred.count()['tconst'] == test.count()['tconst'] )
      if tc1 and tc2 :
        print("PASS: dataframe đầu ra đúng định dạng")
      else:
        print("FAIL: dataframe đầu ra không đúng định dạng, đ/c kiểm tra lại")
      
      tc3 = ( pd.merge(df_pred, test, how='inner', on = 'tconst')['tconst'].count() == test.count()['tconst'])
      
      if tc3 :
        print("PASS: dataframe đầu ra đúng số lượng bản ghi")
      else:
        print("FAIL: dataframe đầu ra không đúng số lượng bản ghi, đ/c kiểm tra lại")
        
    def score_iib(df_pred):
      eval = pd.merge(df_pred, test, how='inner', on = 'tconst' )
      print( roc_auc_score( eval['isAdult'], eval['pred'] ))
    

In [ ]:
spark = pyspark.sql.SparkSession \
    .builder \
    .appName("DS_test") \
    .getOrCreate()

#### DATA: Bộ IMDb

Thông tin bộ dữ liệu IMDb: https://www.imdb.com/interfaces/

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name

![Alt text](https://relational.fit.cvut.cz/assets/img/datasets-generated/imdb_ijs.svg "Schema bộ dữ liệu")

In [ ]:
# Tải dữ liệu về
file_list = ["title.akas.tsv.gz", "title.ratings.tsv.gz", 
             "title.basics.tsv.gz", "title.basics.tsv.gz", "title.crew.tsv.gz", "title.principals.tsv.gz",
            "name.basics.tsv.gz" ]
      
for i in file_list:
    data_downloader(i)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Loaded title.akas.tsv.gz to local path: /content/title.akas.tsv.gz


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Loaded title.ratings.tsv.gz to local path: /content/title.ratings.tsv.gz
Loaded title.basics.tsv.gz to local path: /content/title.basics.tsv.gz


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

Loaded title.crew.tsv.gz to local path: /content/title.crew.tsv.gz


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Loaded title.principals.tsv.gz to local path: /content/title.principals.tsv.gz


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Loaded name.basics.tsv.gz to local path: /content/name.basics.tsv.gz


In [ ]:
# Setting to speed up 
spark.sql("set spark.sql.shuffle.partitions=1")

DataFrame[key: string, value: string]

## PHẦN I. CHUNG
[ 10 điểm ] <br>
Guide: Sử dụng pyspark hoặc spark-SQL thực hiện tất cả các câu thuộc phần này.

**Câu 1**. [1đ] Đọc các file vào pyspark dataframe, biết rằng các cột trong các file dữ liệu đều phân cách nhau bởi dấu tab (\t). Lưu tương ứng các dataframe với các file dữ liệu:
- df_rating : "title.ratings.tsv.gz"
- df_akas  :"title.akas.tsv.gz"
- df_basics  :"title.basics.tsv.gz"
- df_crew  :"title.crew.tsv.gz"
- df_principals :"title.principals.tsv.gz"
- df_person  : "name.basics.tsv.gz"

(Xem mô tả quan trọng của dữ liệu phía trên)

In [ ]:
df_ratings = (
    # YOUR CODE START 
    spark.read.options(header='True', delimiter='\t', nullValue = r'\N').csv("/content/title.ratings.tsv.gz")
    
    # / END YOUR CODE
)

df_akas= (
    # YOUR CODE START 
    spark.read.options(header='True', delimiter='\t', nullValue = r'\N').csv("/content/title.akas.tsv.gz")
    
    # / END YOUR CODE
)

df_basics = (
    # YOUR CODE START 
    spark.read.options(header='True', delimiter='\t', nullValue = r'\N').csv("/content/title.basics.tsv.gz")
    
    # / END YOUR CODE
)

df_crew = (
    # YOUR CODE START 
    spark.read.options(header='True', delimiter='\t', nullValue = r'\N').csv("/content/title.crew.tsv.gz")
    
    # / END YOUR CODE
)

df_principals = (
    # YOUR CODE START 
    spark.read.options(header='True', delimiter='\t', nullValue = r'\N').csv("/content/title.principals.tsv.gz")
    
    # / END YOUR CODE
)

df_person = (
    # YOUR CODE START 
    spark.read.options(header='True', delimiter='\t', nullValue = r'\N').csv("/content/name.basics.tsv.gz")
    
    # / END YOUR CODE
)

In [ ]:
%%time
TEST.test_1(df_ratings)
TEST.test_2(df_akas)

PASS
PASS
CPU times: user 128 ms, sys: 20.3 ms, total: 148 ms
Wall time: 22.1 s


**Câu 2**. [1đ] Tìm top 3 ngôn ngữ có nhiều bộ phim (tất cả nội dung movie, tv show,...) nhất trong bộ dataset này

In [ ]:
%%time
# YOUR CODE START 
df_akas.filter(F.col("language").isNotNull()).groupby("language").count()\
       .orderBy("count", ascending=False)\
       .show(3)
# / END YOUR CODE

+--------+-------+
|language|  count|
+--------+-------+
|      ja|3626718|
|      fr|3594465|
|      hi|3563670|
+--------+-------+
only showing top 3 rows

CPU times: user 232 ms, sys: 35.5 ms, total: 267 ms
Wall time: 48 s


**Câu 3**. [2đ]
Trong các bộ phim Mỹ (region="US", titleType="movie") được công chiếu giai đoạn 2010-2015, tìm top 5 thể loại phim có bình quân rating điểm cao nhất.

In [ ]:
%%time
# YOUR CODE START 
df_genre_pick = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")>=2010) & (F.col("startYear")<=2015) )\
         .withColumn("arr_genres", F.split("genres", ","))\
         .select("tconst", F.explode("arr_genres").alias("genre"))
         
df_film_pick = df_akas.filter(F.col("region")=="US").withColumn('tconst', F.col('titleId'))


df_ratings.join(df_genre_pick, on='tconst').join(df_film_pick, on="tconst")\
    .groupby("genre").agg(F.avg("averageRating").alias("rating"))\
    .orderBy("rating", ascending=False)\
    .show(5)
# / END YOUR CODE

+-----------+-----------------+
|      genre|           rating|
+-----------+-----------------+
|  Game-Show|              8.9|
|      Short|7.339999999999999|
|       News|7.282631578947371|
|Documentary|7.231346309813465|
|  Biography|7.140000000000006|
+-----------+-----------------+
only showing top 5 rows

CPU times: user 388 ms, sys: 71.9 ms, total: 460 ms
Wall time: 1min 23s


**Câu 4**. [3đ] Bạn đặt ra một giả thuyết rằng một đạo diễn có xu hướng hợp tác với các diễn viên thân thuộc với họ, hơn là liên tục tìm kiếm các diễn viên mới cho các phim khác nhau. <br>
Hãy lựa chọn các metrics phù hợp, phân tích từ bộ dữ liệu thực tế và rút ra kết luận cho giả thuyết trên có đúng hay không? <br>
( Phân tích trên thể loại phim titleType="movie", region="US",  thời gian công chiếu trong khoảng 2000-2020)

In [ ]:
%%time
# YOUR CODE START 

"""Nhận xét: nếu các đạo diễn có xu hướng lựa chọn diễn viên thân thuộc là đúng, sẽ có rất nhiều bộ phim có sự hợp tác của họ
=> Vẽ phân phối số lần hợp tác sẽ đánh giá được giả thiết
"""
# Lấy tập data theo yêu cầu đề bài
movie = df_basics.filter((F.col("titleType")=="movie") &  F.col("startYear").between(2010, 2015) )
us = df_akas.filter(F.col("region")=="US").selectExpr('titleId AS tconst')

principals = df_principals.filter( F.col('category').isin('actor', 'actress', 'director') )\
                .join(movie, on='tconst').join(us, on='tconst')

# Đếm số lần hợp tác
actor = principals.filter(F.col('category').isin('actor', 'actress')).withColumn('actor', F.col('nconst'))
director = principals.filter(F.col('category')=='director').withColumn('director', F.col('nconst'))

colaboration = actor.join(director, on='tconst').groupby('actor', 'director').agg(F.count('tconst').alias('movie_num'))\
                .withColumn('is_1time', F.when(F.col('movie_num')==1, 1).otherwise(0))

his = colaboration.groupby('is_1time').count()
print('Số bộ phim: ', actor.join(director, on='tconst').select(F.countDistinct('tconst')).collect())
print('Số lần hợp tác:')
his.show()
# / END YOUR CODE

Số bộ phim:  [Row(count(DISTINCT tconst)=29258)]
Số lần hợp tác:
+--------+------+
|is_1time| count|
+--------+------+
|       1|116545|
|       0| 12365|
+--------+------+

CPU times: user 1.89 s, sys: 332 ms, total: 2.22 s
Wall time: 6min 38s


In [ ]:
your_conclusion = ('''
# YOUR CONCLUSION HERE
Trung bình 1 bộ phim có 100 diễn viên thì sẽ có khoảng 11 diễn viên cũ (từng hợp tác với đạo diễn)
=> Các diễn viên và đạo diễn có xu hướng hợp tác với nhau nhiều lần

''')

**Câu 5**. [3đ] Một giải thưởng về phim (titleType = 'movie') được trao cho các đạo diễn. Danh sách đề cử là các đạo diễn thỏa mãn các điều kiện: <br>
1. Đạo diễn vẫn còn sống
2. Có trên 5 bộ phim hay (phim hay là phim được chấm điểm averageRating >7.5).
3. Tỷ trọng phim tệ (phim tệ là phim có averageRating <= 4) chiếm không quá 20% tổng số phim của đạo diễn đó và không quá 10 bộ phim.

Trong đó, averageRating chỉ được tính khi có trên 200 phiếu bầu. <br>
Giải thưởng được trao cho 3 trong số các đạo diễn thuộc danh sách đề cử. <br>
Đặt p (float) là xác suất để đạo diễn trúng cử đồng thời có kiêm biên kịch (writer, xác định bởi primaryProfession).  <br>
Hãy tính p.

In [ ]:
%%time
# YOUR CODE START 
film = df_basics.filter(F.col("titleType")=="movie").select("tconst")
director = df_person.filter(F.col("deathYear").isNull()).select('nconst', 'primaryProfession')
ratings = df_ratings.filter(F.col("numVotes")>200)

principals = df_principals.filter( F.col('category')=='director' ).select('nconst', 'tconst')

matched = ratings.join(film, on='tconst')\
                .join(principals, on='tconst')\
                .join(director, on='nconst')\
                .withColumn('is_bad', F.when(F.col('averageRating') <=4, 1).otherwise(0))\
                .withColumn('is_good', F.when(F.col('averageRating') >7.5, 1).otherwise(0))\
                .withColumn('is_writer', F.when(F.col('primaryProfession').contains('writer'), 1).otherwise(0) ).persist()

smr_by_director = matched.groupby("nconst", "is_writer").agg(
    F.sum("is_bad").alias('bad'),
    F.sum("is_good").alias('good'),
    F.count("tconst").alias('total'),
    (F.sum("is_bad")/F.count("tconst")).alias("bad_ratio")
)
smr = smr_by_director.filter(
    (F.col('bad')<=10 ) &
    (F.col('good')>5) &
    (F.col('bad_ratio')<=0.2)
).selectExpr("COUNT(1) AS dir_num", 
             "SUM(is_writer) AS writer", 
             "SUM(is_writer)/COUNT(1) AS pct" 
             ).collect()
writer_num = smr[0].writer
dir_num = smr[0].dir_num


"""Với số đạo diễn đề cử là dir_num, số đạo diễn có kiêm biên kịch là writer_num
Số cách chọn 3 đạo diễn từ writer_num là x = C(writer_num, 3)
Số cách chọn 3 đạo diễn từ dir_num là y = C(dir_num, 3)
Xác suất để 3 đạo diễn là đồng thời là biên kịch là: p = x/y
"""

import math

def comb(n,r):
    f = math.factorial
    return f(n) / f(r) / f(n-r)

p = comb(writer_num, 3)/ comb(dir_num,3)

print(p)

# / END YOUR CODE

0.49418604651162795
CPU times: user 604 ms, sys: 98.6 ms, total: 703 ms
Wall time: 2min 5s


### PHẦN II. TỰ CHỌN
[ 10 điểm ] <br>
DS chọn 1 trong 2 câu II.a hoặc II.b
**Chỉ sử dụng dữ liệu train đã được chia sẵn để huấn luyện model, không sử dụng tập test.** <br>
Có thể sử dụng tập test để đánh giá độ chính xác của model.

Điểm số được chấm dựa trên độ chính xác của model trên tập test.

**Câu II.a**. Trên môi trường pyspark, hãy tổng hợp dữ liệu và huấn luyện một mô hình dùng pyspark MLlib dự đoán averageRating

In [ ]:
# CHẠY ĐOẠN CODE SAU
train_film = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")>=2000) & (F.col("startYear")<=2015) ).select("tconst")
test_film  = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")> 2015) & (F.col("startYear")<=2021) ).select("tconst")

train_ratings = df_ratings.join( train_film, on = 'tconst')
test_ratings = df_ratings.join( test_film, on = 'tconst')

Chỉ sử dụng thông tin của các bộ phim trong train_film (và nhãn là averageRating trong train_ratings) để xây dựng model. <br>
Có thể join thêm với các thông tin ở các bảng khác không giới hạn để bổ sung feature. <br>
Metric đánh giá là MAE

**YÊU CẦU ĐẦU RA** <br>
Dataframe: df_pred {tconst: string, pred: float}, trong đó tconst là các mã bộ phim tương ứng với các bộ phim trong test_film, pred là dự đoán averageRating của model.

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor

In [ ]:
def array_to_onehot(df, inputCol, sep=',', id_col = 'tconst'):
  df = df.withColumn('arr_'+inputCol, F.split(inputCol, sep))\
          .select(id_col, F.explode('arr_'+inputCol).alias("element_"+inputCol))\
          .withColumn('value', F.lit(1))
  df_vec = df.groupby(id_col).pivot("element_"+inputCol).count()
  return df_vec


In [ ]:
def cast_col(df, col, dtype='int'):
  df = df.withColumnRenamed(col, col+'tmp')
  df = df.withColumn(col, F.col(col+'tmp').cast(dtype))
  return df.drop(col+'tmp')
def cast_cols(df, cols, dtype='int'):
  for col in cols:
    df = cast_col(df, col)
  return df

In [ ]:
# YOUR CODE START 
akas = df_akas.groupby('titleId').agg(F.count('titleId').alias('region_num'))

full = df_basics.filter((F.col("titleType")=="movie") & F.col("startYear").between(2000,2021))\
        .join(akas, on = [df_basics.tconst == df_akas.titleId], how='left')\
        .join(df_ratings, on='tconst', how='left')\
        .withColumn('dataset', F.when(F.col("startYear").between(2000,2015), 'train').otherwise('test'))

num_cols = ['isAdult','startYear',  'runtimeMinutes', 'region_num', 'averageRating']
full = cast_cols(full, num_cols, dtype='double')

# Optional: adding genre information to features
df_genre = array_to_onehot(full, inputCol='genres')
full = full.join(df_genre, on='tconst', how='left').fillna(0)
cols_genres = df_genre.drop('tconst').columns

pipe = Pipeline(stages=[
        pyspark.ml.feature.VectorAssembler(inputCols= num_cols+cols_genres, outputCol='fts'),
        RandomForestRegressor( featuresCol = 'fts', labelCol = 'averageRating', predictionCol='pred',  
                              numTrees=10 )
])

model = pipe.fit(full.filter(F.col('dataset')=='train'))

df_pred = model.transform(full.filter(F.col('dataset')=='test')).select('tconst', 'pred')


# / END YOUR CODE

In [ ]:
# df_pred hợp lệ khi testcase báo 2 thông báo PASS như sau:
#>> PASS: dataframe đầu ra đúng định dạng
#>> PASS: dataframe đầu ra đúng số lượng bản ghi
TEST.iia_schema( df_pred )

FAIL: dataframe đầu ra không đúng định dạng, đ/c kiểm tra lại
PASS: dataframe đầu ra đúng số lượng bản ghi


In [ ]:
# Tính MAE
TEST.mae_iia( df_pred )

+----------------+
|             mae|
+----------------+
|0.83781821998956|
+----------------+



**Câu II.b**. Bạn đưa ra giả thuyết thông tin về tiêu đề phim có thể giúp phân loại phim đó có phải là phim không dành cho trẻ nhỏ hay không. <br>
Chỉ sử dụng thông tin từ bảng title.basics.tsv.gz, hãy xây dựng mô hình dự đoán nhãn isAdult. Trong đó, bắt buộc có sử dụng xử lý chuỗi thông tin tiêu đề phim originalTitle.

(Mô hình có thể được xây dựng bằng pyspark hoặc python)

In [ ]:
# CHẠY ĐOẠN CODE SAU
train = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")>=2000) & (F.col("startYear")<=2015) ).toPandas()
test = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")> 2015) & (F.col("startYear")<=2021) ).toPandas()

In [ ]:
train['tconst'].count()

161584

Có thể sử dụng model pyspark hoặc python. <br>
Metric đánh giá là accuracy

**YÊU CẦU ĐẦU RA** <br>
- Mô hình: có các bước biến đổi dữ liệu và xử lý text
- Dữ liệu: pandas dataframe df_pred {tconst: string, pred: int}, trong đó tconst là các mã bộ phim tương ứng với các bộ phim trong test, pred là dự đoán isAdult của model, pred phải mang giá trị 0 hoặc 1.

In [ ]:
# YOUR CODE START 
# DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

sentences = []
labels = []

# train 
train_sentences = train['originalTitle']
valid_sentences = test['originalTitle']
# label 
train_labels = train['isAdult']
valid_labels = test['isAdult']

token = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token= oov_tok)
token.fit_on_texts(train_sentences)
word_index = token.word_index

train_sequences = token.texts_to_sequences(train_sentences)
valid_sequences = token.texts_to_sequences(valid_sentences)

train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, truncating=trunc_type, padding=padding_type, maxlen=max_length)
valid_padded = tf.keras.preprocessing.sequence.pad_sequences(valid_sequences, truncating=trunc_type, padding=padding_type, maxlen=max_length)

train_labels = np.asarray(train_labels).astype('int')
valid_labels = np.asarray(valid_labels).astype('int')

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_padded, train_labels, epochs=3, batch_size=1024)

prd = model.predict(valid_padded)
df_pred = pd.DataFrame({'tconst':test.tconst, 'pred': prd.reshape(prd.shape[0])})

# / END YOUR CODE

Epoch 1/3
2525/2525 [==============================] - 202s 79ms/step - loss: 0.0358 - accuracy: 0.9947
Epoch 2/3
2525/2525 [==============================] - 206s 82ms/step - loss: 0.0285 - accuracy: 0.9949
Epoch 3/3
2525/2525 [==============================] - 200s 79ms/step - loss: 0.0273 - accuracy: 0.9950


In [ ]:
# df_pred hợp lệ khi testcase báo 2 thông báo PASS như sau:
#>> PASS: dataframe đầu ra đúng định dạng
#>> PASS: dataframe đầu ra đúng số lượng bản ghi
TEST.iib_schema(df_pred)

PASS: dataframe đầu ra đúng định dạng
PASS: dataframe đầu ra đúng số lượng bản ghi


In [ ]:
# Tính AUC
TEST.score_iib(df_pred)

0.7530125817868337


--- YOUR TEST END HERE ---